In [1]:
#@title Imports 
import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
DISPLAY_SORTED = True
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib import ticker
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from matplotlib import animation, rc

GIT = 'https://github.com/tyokokur/tmpdat/raw/main/sysg3/'

In [ ]:
from urllib.error import HTTPError
fname = GIT+'ph_a020x075_Lxy5.dat' #"C:/Users/takas/Downloads/ph_omp.dat"
block_Nik = [1]
Nx = 20
Ny = 20
Nz = 300
NxNyNz = Nx*Ny*Nz

a = 1
b = 1
c = 1
names = ['rx', 'ry', 'rz', 'phA']
for j in range(len(block_Nik)):
    names += ['phA_T{:d}'.format(a)] +  ['*'*b]+ ['ph{:d}'.format(c+i) for i in range(block_Nik[j])] + ['*'*(b+1)]
    a += 1
    b += 2
    c += block_Nik[j]

names += ['phB']
try: df = pd.read_csv(fname, sep="\s+", skiprows=0, names=names)
except HTTPError: 
    print('{} not found'.format(fname))

if (df.shape[0] != NxNyNz): print('DIMS WRONG; df: {}, NxNyNz: {}'.format(df.shape[0], NxNyNz))

for i in range(len(block_Nik)*2):
    names.remove('*'*(i+1))
    df = df.drop('*'*(i+1),axis=1)
    
rows = range(0, NxNyNz)
phAr = pd.DataFrame(data=None, index=rows, columns=range(4), dtype=None, copy=False).fillna(0)
phAT = pd.DataFrame(data=None, index=rows, columns=range(1+len(block_Nik)), dtype=None, copy=False).fillna(0)
ph1r = pd.DataFrame(data=None, index=rows, columns=range(1+sum(block_Nik)), dtype=None, copy=False).fillna(0)

phB = pd.DataFrame(data=None, index=rows, columns=range(4), dtype=None, copy=False).fillna(0)

ph1r = df.loc[:, ['ph{:d}'.format(i+1) for i in range(sum(block_Nik))]]
ph1r.insert(0, 'rx', df.iloc[:,0])
ph1r.insert(1, 'ry', df.iloc[:,1])
ph1r.insert(2, 'rz', df.iloc[:,2])

phAT = df.loc[:, ['phA_T{:d}'.format(i+1) for i in range(len(block_Nik))]]
phAT.insert(0, 'rx', df.iloc[:,0])   
phAT.insert(1, 'ry', df.iloc[:,1])   
phAT.insert(2, 'rz', df.iloc[:,2])   

print(NxNyNz)
for i in range(NxNyNz):
    if (i % 100000==0): print(i, end = ' ')
    phAr.iloc[i, 0] = df.iloc[i,0]
    phAr.iloc[i, 1] = df.iloc[i,1]
    phAr.iloc[i, 2] = df.iloc[i,2]
    phAr.iloc[i, 3] = df.iloc[i,3] 
    # phB.iloc[i, 0] = df.iloc[i, 0]
    # phB.iloc[i, 1] = df.iloc[i, 1]
    # phB.iloc[i, 2] = df.iloc[i, 2]
    # phB.iloc[i, 3] = df.iloc[i, 4 + sum(block_Nik) + len(block_Nik)] 
print('Done')

120000
0 

In [ ]:
X, Y, Z = np.array(phAr.iloc[:,0]), np.array(phAr.iloc[:,1]), np.array(phAr.iloc[:,2])
vol = np.array(phAr.iloc[:,3])
if block_Nik[0] > 1: bvols= [np.array(ph1r.iloc[:,i+3]) for i in range(block_Nik[0])]

Lz_max = 40
vol   =  vol[Z < Lz_max]
if block_Nik[0] > 1: bvols = [bvols[i][Z < Lz_max] for i in range(block_Nik[0])]
X = X[Z < Lz_max]
Y = Y[Z < Lz_max]
Z = Z[Z < Lz_max]

Nz_new = int(len(Z)/Nx/Ny)
x = [X[i*Ny*Nz_new] for i in range(Nx)]
y = [Y[j*Nz_new]    for j in range(Ny)]
z = [Z[k]           for k in range(Nz_new)]

xslice = 1
islice = int(xslice*(x[1]-x[0]))
vol  =  np.array([  vol[(islice*Ny+j)*Nz_new+k] for j in range(Ny) for k in range(Nz_new)]).reshape(Ny, Nz_new)
if block_Nik[0] > 1: bvols= [np.array([bvols[i][(islice*Ny+j)*Nz_new+k] for j in range(Ny) for k in range(Nz_new)]).reshape(Ny, Nz_new) for i in range(block_Nik[0])]

import plotly.graph_objects as go
heatmap = go.Heatmap(
            x=z, y=y, z=vol,
            zmin = 0.0, zmax = 0.65,
            colorscale= 'Picnic'
          )
overall = go.Scatter(
            x = z,
            y = 5.0*vol[int(Ny/2), :] + max(y)/2-1.3,
            line = dict(color='black', width=3)
          )
if block_Nik[0] > 1: blocks = [go.Scatter(
                                x = z,
                                y = 1.5*bvols[i][int(Ny/2),:] + max(y)/2-1,
                                line = dict(color='black', width=3, dash='dot')
                              ) for i in range(block_Nik[0])]

fname = GIT+'ph_a020x075_1D.dat'
df = pd.read_csv(fname, sep="\s+", skiprows=0)
a000x075_1d = pd.DataFrame([df.iloc[:,0], df.iloc[:,1]], index=['z', 'pha']).T

reference = go.Scatter(
                x = a000x075_1d.z,
                y = 1.5*a000x075_1d.pha + max(y)/2-1,
                line = dict(color='black', width=3, dash='dash')
            )

layout = go.Layout( xaxis_range = [0, Lz_max], yaxis_range = [0, y[-1]],
                    yaxis_title = r'$\text{y [nm] /  } \phi\ \text{(scaled)}$', xaxis_title = 'z [nm]',
                    scene_xaxis_showticklabels=True,
                    scene_yaxis_showticklabels=True,
                    scene_zaxis_showticklabels=True,
                    showlegend = False,
                    template='plotly_white', width=900, height=600)
if block_Nik[0] > 1: fig = go.Figure(data = [heatmap, overall, *blocks], layout = layout)
else: fig = go.Figure(data = [heatmap, overall], layout = layout)
# if block_Nik[0] > 1: fig = go.Figure(data = [heatmap, overall, *blocks, reference], layout = layout)
# else: fig = go.Figure(data = [heatmap, overall, reference], layout = layout)
fig.show()

In [ ]:
X, Y, Z = np.array(phAr.iloc[:,0]), np.array(phAr.iloc[:,1]), np.array(phAr.iloc[:,2])
vol = np.array(phAr.iloc[:,3])

Lz_max = 25
vol = vol[Z < Lz_max]
X = X[Z < Lz_max]
Y = Y[Z < Lz_max]
Z = Z[Z < Lz_max]

import plotly.graph_objects as go
fig = go.Figure(data=go.Volume(
    x=X, y=Y, z=Z,
    value=vol,
    isomin  =0.0,
    isomax  =0.65,
    opacity =0.75,
    surface_count=20,
    colorscale= 'Picnic' #Blackbody,Bluered,Blues,Cividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portland,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd.
    ))
fig.update_layout(scene_xaxis_showticklabels=True,
                  scene_yaxis_showticklabels=True,
                  scene_zaxis_showticklabels=True,
                  scene_aspectmode='data',
                  template='plotly_white', width=900, height=600)
fig.show()